In [1]:
import time

import requests
from bs4 import BeautifulSoup

https://secure.runescape.com/m=itemdb_rs/top100?list=0

In [2]:
class RS_ITEM:
    def __init__(self, item=None):
        if item is None:
            self.item = {}
        else:
            self.item = item

    def __repr__(self):
        return f"{self.item}"

    @staticmethod
    def fromSearchHTMLRow(soup):
        c = RS_ITEM()
        tds = soup.find_all("td")
        d = tds[2]
        c.item = {
            "name": tds[0].find("a").get("title").strip(),
            "price": int(
                d.find("a")
                .text.strip()
                .replace(",", "")
                .replace(".", "")
                .replace("k", "00")
            ),
            "url": d.find("a").get("href"),
            "itemid": d.find("a").get("href").split("?")[-1].split("=")[-1],
        }
        return c

In [3]:
def search(query, page=None):
    items = []

    if page == None:
        res = requests.post(
            "https://secure.runescape.com/m=itemdb_rs/results#main-search",
            {"query": query},
        )
    else:
        res = requests.get(f"https://secure.runescape.com/m=itemdb_rs/results{page}")

    soup = BeautifulSoup(res.text)
    table = soup.find("table", {"class": "results-table"})
    tbody = table.find("tbody")
    # print(table)
    for row in tbody.find_all("tr"):
        items.append(RS_ITEM.fromSearchHTMLRow(row))

    if page == None:
        pages = soup.find("div", {"class": "pagination"}).find_all("li")
        if len(pages) > 1:
            for page in range(1, len(pages)):
                items.extend(search(query, page=pages[page].find("a").get("href")))

    return items

In [4]:
search("Orichalcite Ore")

In [5]:
search("Drakolith")

In [6]:
ori = search("Orikalkum")
for item in ori:
    if "+ 3" in item.item["name"]:
        print(item)

In [7]:
search("notepaper")

In [8]:
search("chronotes")

In [9]:
226 * 2000

In [10]:
search("Necronium battleaxe")